Copyright **`(c)`** 2023 Thomas Baracco `<s308722@studenti.polito.it>`  
[`https://github.com/baraccothomas/computational-intelligence`](https://github.com/baraccothomas/computational-intelligence)  
Free for personal or classroom use.

In [109]:
import numpy as np

from random import random
from collections import namedtuple
from functools import reduce
from queue import PriorityQueue, SimpleQueue, LifoQueue


In [110]:
PROBLEM_SIZE = 5
NUM_SETS = 10
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

State = namedtuple('State', ['taken', 'not_taken'])

In [111]:
class Node:
    def __init__(self, state, g, h):
        self.state = state
        self.g = g
        self.h = h
    
    def f(self):
        return self.g + self.h

    def __str__(self):
        return f"State: {self.state} | g = {self.g} | h = {self.h} | f = {self.f()}"

In [112]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for i in range(PROBLEM_SIZE)]),
        )
    )


def h(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

In [113]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [114]:
openList = []
closeList = []
initialState = State(set(), set(range(NUM_SETS)))
openList.append(Node(initialState, 0, h(initialState)))

counter = 0
parentNode = {}
current_node = openList[0]
closeList.append((current_node, parentNode))
openList.remove(current_node)

while not goal_check(current_node.state):
    counter += 1
    for action in current_node.state[1]:
        new_state = State(
            current_node.state.taken ^ {action},
            current_node.state.not_taken ^ {action},
        )
        openList.append(Node(new_state, current_node.g + 1, current_node.h + h(new_state)))
    openList.sort(key=lambda x: x.f())

    
    closeList.append((openList[0], current_node))
    current_node = openList[0]
    openList.remove(current_node)
    
print(current_node.state)

State(taken={2, 3}, not_taken={0, 1, 4, 5, 6, 7, 8, 9})
